# The AprilTag system: 6DoF vision with monocular cameras


The library Apriltag allows us to detect markers in images, and to compute the relative pose between the camera coordinate frame and the tag.

We will need Apriltag, OpenCV-python, and some NumPy operations to proceed. Install them with

```terminal
python -m pip install apriltag
python -m pip install opencv-python
```
Then import them to your python project:

In [1]:
import apriltag
import cv2
import numpy as np

## Detecting apriltags in images

First we need to read an image into memory, and store it as greyscale

In [2]:
image = cv2.imread('./data/skew.jpeg', cv2.IMREAD_GRAYSCALE)

cv2.imshow("Image view", image)
cv2.waitKey(1000) # waits until a key is pressed


-1

Then we need an apriltag detector

In [3]:
detector = apriltag.Detector()


Now we can process the image, and see what comes out of the Apriltag detector:

In [4]:
detections = detector.detect(image)

print('detections = \n', detections)

detections = 
 [Detection(tag_family=b'tag36h11', tag_id=5, hamming=0, goodness=0.0, decision_margin=87.4375, homography=array([[ 9.28986229e-01,  2.25901040e-01,  2.52270766e+00],
       [-1.91712313e-01,  2.44261297e-01,  1.57486813e+00],
       [ 1.30679241e-03, -1.56725245e-03,  1.87352580e-02]]), center=array([134.65027603,  84.05905775]), corners=array([[ 72.0067749 ,  80.14012146],
       [149.27796936,  52.70389938],
       [199.06008911,  88.08849335],
       [114.72151947, 126.77729797]]))]


So we've processed an image, and discovered one Tag, labelled with the ID=5.

We have some more information regarding this detection, which we can access easily:

In [5]:

for detection in detections:  

    print('tag #',              detection.tag_id)
    print('detection hamming',  detection.hamming)
    print('detection goodness', detection.goodness)
    print('detection margin',   detection.decision_margin)
    print('homography\n',       detection.homography)
    print('tag center',         detection.center)
    print('tag corners\n',      detection.corners)


tag # 5
detection hamming 0
detection goodness 0.0
detection margin 87.4375
homography
 [[ 9.28986229e-01  2.25901040e-01  2.52270766e+00]
 [-1.91712313e-01  2.44261297e-01  1.57486813e+00]
 [ 1.30679241e-03 -1.56725245e-03  1.87352580e-02]]
tag center [134.65027603  84.05905775]
tag corners
 [[ 72.0067749   80.14012146]
 [149.27796936  52.70389938]
 [199.06008911  88.08849335]
 [114.72151947 126.77729797]]


These fields are documented in Apriltag as follows

    # The decoded ID of the tag
    tag_id

    # How many error bits were corrected? Note: accepting large numbers of
    # corrected errors leads to greatly increased false positive rates.
    # NOTE: As of this implementation, the detector cannot detect tags with
    # a hamming distance greater than 2.
    hamming

    # A measure of the quality of tag localization: measures the
    # average contrast of the pixels around the border of the
    # tag. refine_pose must be enabled, or this field will be zero.
    goodness

    # A measure of the quality of the binary decoding process: the
    # average difference between the intensity of a data bit versus
    # the decision threshold. Higher numbers roughly indicate better
    # decodes. This is a reasonable measure of detection accuracy
    # only for very small tags-- not effective for larger tags (where
    # we could have sampled anywhere within a bit cell and still
    # gotten a good detection.)
    decision_margin

    # The 3x3 homography matrix describing the projection from an
    # "ideal" tag (with corners at (-1,-1), (1,-1), (1,1), and (-1,
    # 1)) to pixels in the image. This matrix will be freed by
    # apriltag_detection_destroy.
    homography;

    # The center of the detection in image pixel coordinates.
    center;

    # The corners of the tag in image pixel coordinates. These always
    # wrap counter-clock wise around the tag.
    corners;

## Computing camera-to-tag relative pose

Camera-to-tag transforms can be obtained from the detected corners if we know
  - the geometry of the tags
  - the geometry of the camera (intrinsic and distortion parameters)

So let us define these parameters:

In [6]:
# Tag geometry
tag_size    = 0.2
tag_corners = tag_size / 2 * np.array([[-1,1,0],[1,1,0],[1,-1,0],[-1,-1,0]])

# Camera calibration
K           = np.array([[   419.53, 0.0,    427.88  ], 
                        [   0.0,    419.53, 241.32  ], 
                        [   0.0,    0.0,    1.0     ]])  
# warning: these params do not corresopnd to the ones of the camera used to take the image skew.jpeg

distortion_model = np.array([])  # we assume rectified images, therefore with no distortion


From here there are two ways of computing the relative pose between the camera and the tag.

- One method uses the homography provided by the detector to extract translation `T` and rotation `R`. This method is unstable and we do not recommend it.

- The other method uses the PnP algorithm which, given the four corners of the tag in tag reference (which are known), the projected same corners in the image obtained by the detector, and the camera calibration parameters, computes the transformation (`T`,`R`) between camera and tag. 

We use OpenCV for this, and recover a translation vector and a rotation vector:

In [7]:
(_, rotation_vector, translation_vector) = cv2.solvePnP(tag_corners, detection.corners, K, distortion_model, flags = cv2.SOLVEPNP_IPPE_SQUARE)

T = translation_vector
w = rotation_vector

print('T = \n', T)
print('w = \n', w)


T = 
 [[-0.71469273]
 [-0.38025099]
 [ 1.02429824]]
w = 
 [[ 2.39555008]
 [-1.05666679]
 [ 0.51011416]]


See that although `T` and `w` are vectors, they come represented as 2-dimensional arrays. We can clean them up to avoid trouble down the road:

In [8]:
T = (T.T)[0]
w = (w.T)[0]

print('T = \n', T)
print('w = \n', w)


T = 
 [-0.71469273 -0.38025099  1.02429824]
w = 
 [ 2.39555008 -1.05666679  0.51011416]


It is now important to know how to interpret these T and R. Are they camera-to-tag? Or tag-to-camera? Reading the doc of cv2.computePnP(), we see that they transform points in tag frame into points in camera frame. Let us rename the variables to account for this and be more verbose:

In [9]:
T_c_t = T
w_c_t = w

In order to obtain a rotation matrix from a rotation vector, we require the exponential in SO(3), or the Rodrigues formula. We use pinocchio for this:

In [10]:
import pinocchio as pin

R_c_t = pin.exp(w_c_t)

print('R = \n',R_c_t)

R = 
 [[ 0.63436322 -0.75956485  0.14368208]
 [-0.58494682 -0.59316663 -0.55316414]
 [ 0.50539145  0.26686061 -0.82058814]]


We can now reproject the tag corners into the image, and see if they differ much from the detections:

In [11]:
projected_corners = cv2.projectPoints(tag_corners, R_c_t, T_c_t, K, distortion_model)

print('projected corners\n', projected_corners[0])
print()
print('detected corners\n', detection.corners)

projected corners
 [[[ 69.72493078  81.51959134]]

 [[150.91120302  51.62626837]]

 [[197.61212802  89.45084717]]

 [[116.83486974 125.06570759]]]

detected corners
 [[ 72.0067749   80.14012146]
 [149.27796936  52.70389938]
 [199.06008911  88.08849335]
 [114.72151947 126.77729797]]


We see that they kind of match, but not really. This is because the image used was taken from the internet, and we do not have the correct camera calibration parameters.

Let us then re-do the whole process with a proper image taken by the camera whose parameters we know:

In [12]:
image = cv2.imread('./data/visual_odom_laas_corridor/short2/frame0000.jpg', cv2.IMREAD_GRAYSCALE)
cv2.imshow("Image view", image)
cv2.waitKey(10) # waits so the image can be drawn (??)

detections = detector.detect(image)

for detection in detections[0:2]:   # we'll show results of only 2 detections

    print('Tag # ', detection.tag_id)

    (_, w_c_t, T_c_t) = cv2.solvePnP(tag_corners, detection.corners, K, distortion_model, flags = cv2.SOLVEPNP_IPPE_SQUARE)
    R_c_t = pin.exp(w_c_t)

    projected_corners = cv2.projectPoints(tag_corners, R_c_t, T_c_t, K, distortion_model)

    print('projected corners\n', projected_corners[0])
    print()
    print('detected corners\n', detection.corners)

Tag #  11
projected corners
 [[[530.33061847 324.85984592]]

 [[562.64324848 372.42973975]]

 [[513.0002665  382.6174256 ]]

 [[483.49205875 331.01970981]]]

detected corners
 [[530.2746582  324.92544556]
 [562.69659424 372.34301758]
 [512.9286499  382.71353149]
 [483.56466675 330.94699097]]
Tag #  23
projected corners
 [[[101.27210258 214.14114391]]

 [[175.20315405 222.45660138]]

 [[155.42740652 264.68101411]]

 [[ 70.584094   257.57751718]]]

detected corners
 [[101.36871338 214.23381042]
 [175.09712219 222.36929321]
 [155.53752136 264.78646851]
 [ 70.48442841 257.46499634]]


## Assessing tag detection quality

The quality of a detection can be assessed with different metrics. We provide here some clues:

- Use the `detection.goodness` result
- Use the `detection.hamming` result
- Use the `detection.detection_margin` result
- Use the reprojection error of the corners. That is, the norm of the difference between `projected_corners` and `detected_corners` above. 

We do not provide further insight here, but you may explore these possibilities should your SLAM algorithm show signs of fragility.